# Togetherflow
**Emergent agent motion dynamics in immersive rooms**

In this notebook, we implement Togetherflow, a computational cognitive model that characterizes the motion pattern of human agents in immersive rooms.

In [204]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [205]:
import numpy as np
import matplotlib.pyplot as plt
# from functools import partial

np.set_printoptions(suppress=True)

In [206]:
import tensorflow as tf
import bayesflow as bf
from bayesflow.simulation import Prior, Simulator, GenerativeModel

In [220]:
from initializations import initialize_agents, initialize_beacons
from simulations import motion_simulation

from priors import (
    complete_pooling_prior
)

## Initializations

First, we need to initialize some agents and some beacons.

In [208]:
agent_positions, agent_rotations = initialize_agents()
beacon_positions = initialize_beacons()
agent_rotations.shape

(12, 1)

## Influences

There are 4 influence vectors to an agent:
* **Positional influence** (individual): drift-diffusion process for agent displacement (locomotion);
* **Rotational influence** (individual): drift-diffusion process for agent orientation (head movement);
* **Cohesion influence** (collective): active particle dynamics for agent position following;
* **Alignment influence** (collective): active particle dynamics for agent orientation following.

The implementation of all influences can be found in the `influences` module.

## Final simulation

Combining all influences yields the final simulation.

In [223]:
sim_positions, sim_rotations = motion_simulation(
    agent_positions=agent_positions, 
    agent_rotations=agent_rotations, 
    beacon_positions=beacon_positions
)

sim_positions

array([[[-2.5423832 ,  1.5869689 ],
        [ 1.1488757 ,  1.8642533 ],
        [ 1.906488  , -2.5600734 ],
        ...,
        [ 1.3689766 ,  2.3151743 ],
        [ 1.9968624 , -0.6965947 ],
        [ 0.8874407 ,  0.31693697]],

       [[-2.5337138 ,  1.5956382 ],
        [ 1.2249972 ,  1.9403747 ],
        [ 1.9607716 , -2.5057898 ],
        ...,
        [ 1.33085   ,  2.2770479 ],
        [ 2.057854  , -0.63560325],
        [ 0.95112205,  0.38061833]],

       [[-2.5237892 ,  1.6055628 ],
        [ 1.3025787 ,  2.0179563 ],
        [ 2.0156531 , -2.4509082 ],
        ...,
        [ 1.2948837 ,  2.2410815 ],
        [ 2.1189013 , -0.57455593],
        [ 1.0132352 ,  0.44273147]],

       ...,

       [[-2.0750568 ,  2.0542958 ],
        [ 1.2572119 ,  1.9725897 ],
        [ 6.5204296 ,  2.0538547 ],
        ...,
        [ 1.1217018 ,  2.0679026 ],
        [ 4.7323895 ,  2.038936  ],
        [ 2.6775184 ,  2.107015  ]],

       [[-2.0757897 ,  2.0535629 ],
        [ 1.3371881 ,  2.05

# Priors

In [ ]:
param_names = [r"$w$", r"$r$", r"$v$"]

# Configurator

In [ ]:
def configurator(input_dict: dict = None, transpose: bool = True):
    
    output_dict = {}
    output_dict['parameters'] = input_dict['prior_draws'].astype(np.float32)
    x = input_dict['sim_data'] / 10. 
    if transpose:
        x = np.transpose(x, (0, 2, 1, 3))
    output_dict['summary_conditions'] = x.astype(np.float32)
    return output_dict

# Neural Approximator

In [ ]:
# This one generalizes over different numbers of agents
summary_net = bf.summary_networks.HierarchicalNetwork([
    tf.keras.layers.TimeDistributed(tf.keras.layers.LSTM(units=128)),
    bf.networks.SetTransformer(num_inducing_points=None, input_dim=128, summary_dim=64)
])

inference_net = bf.inference_networks.InvertibleNetwork(
    num_params=3, 
    num_coupling_layers=6,
    coupling_design="affine",
    coupling_settings={
        'kernel_regularizer': None,
        'dropout_prob': 0.0
    }
)

amortizer = bf.amortizers.AmortizedPosterior(
    summary_net=summary_net, 
    inference_net=inference_net
)

# Training

In [ ]:
# trainer = bf.trainers.Trainer(
#     amortizer=amortizer,
#     generative_model=model,
#     configurator=configurator
# )

### Offline training

To make sure that the model is trainable.

In [ ]:
# training_set = model(1000) 

In [ ]:
# losses = trainer.train_offline(training_set, epochs=100, batch_size=100, validation_sims=model(200))